In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
import pickle
import jieba

Duplicate key in file WindowsPath('c:/Users/USER/anaconda3/envs/python3.9.15/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 273 ('font.monospace:  DejaVu Sans Mono, Bitstream Vera Sans Mono, Computer Modern Typewriter, Andale Mono, Nimbus Mono L, Courier New, Courier, Fixed, Terminal, monospace')


In [2]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
chapter_items_seq = pd.read_csv('./data/course_chapter_items_sequence.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [3]:
course_df.head(2)

,course_id,course_name,course_price,teacher_id,teacher_intro,groups,sub_groups,topics,course_published_at_local,description,will_learn,required_tools,recommended_background,target_group
0,61888e868f154b000781b45a,少女人妻華麗變身：七大妝容七彩的夢幻樂園,1800,61888e7bb7fe1c0006850eff,在美妝 KOL 圈裡屬個人風格強烈的 Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影...,生活品味,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",NaN,"<img src=""https://images.api.hahow.in/images/6...",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,所需工具為：視課程實際會用到的彩妝用品,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️ 雖然課程當中會帶到相關彩妝技巧，不...,熱愛彩妝的人
1,54d5a117065a7e0e00725ac0,幾何圖形分割 X 色塊組合,100,54d5a079065a7e0e00725abe,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",2015-03-27 23:38:27.195,"<img src=""https://storage.googleapis.com/hahow...",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,"Adobe Illustrator（必備）, camera",知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧


# 課程 Vectorize

In [4]:
id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)


728

In [5]:
chapter_items_seq

course_chapters = {}
for course_id, chapter_item_name_seq in zip(chapter_items_seq["course_id"], chapter_items_seq["chapter_item_name_seq"]):
    chapter_item_name_seq = chapter_item_name_seq.replace('[', "")
    chapter_item_name_seq = chapter_item_name_seq.replace(']', "")
    chapter_item_name_seq = chapter_item_name_seq.replace('\'', "")
    chapter_item_name_seq = chapter_item_name_seq.replace(' ', "")
    print(chapter_item_name_seq.split(','))
    course_chapters[course_id] = course_chapters.setdefault(course_id, []) + chapter_item_name_seq.split(',')

['拉曲線', '分割工具', '油漆桶上色工具', '幾何圖形分割X色塊組合']
['作品範例', '素材來源與製作', '去背方式', '畫面配置-背景與主題', '畫面配置-細節', '拼貼應用', '一起回到童年']
['課外延伸閱讀(必看)', 'Photoshop基本功能說明-1', 'Photoshop基本功能說明-2', '手繪示範', '插畫繪製-1', '插畫繪製-2', '檔案完稿製作補充說明', '作業介紹']
['前言', '藝術的起源', '藝術史的架構', '埃及', '希臘羅馬時代', '中古世紀', '古典時期的大概念', '文藝復興', '矯飾主義', '巴洛克時期', '17世紀荷蘭', '洛可可', '新古典主義', '浪漫主義', '現實主義', '印象派', '塞尚', '現代美術簡介與結尾', '與藝術對話']
['下載免費軟體virtualdj', 'VirtualDJ音效卡介面設定', 'VDJ介面介紹（畫面下半部）', '介紹介面（畫面上半部）及主要功能介紹', '為何要選用Extended版本', '嘻哈歌曲如何算拍子、Extended歌曲介紹、itunes找素材', '兩首extended歌曲接歌實作1', '兩首extended歌曲接歌實作2', '總結：作業製作方法', '作業示範（如何把三首ExtendedMix接在一起，並玩出一些變化）']
['你的表達，暴露了你的內心訊息！', '課後作業-檢視自己的說話方式', '避免面試ＮＧ的五大地雷', '開啟說話雷達，面試前你應該要知道的事！', '提升說話自信的三大準備']
['培養精準呈現的說話體質', '影響精準表達的四元素：時間、對象、目標、架構', '累積你的全方位說話素材', '整合你的知識內容', '善用三大肢體語言，提升舞台表達力']
['聲音表情萬花筒：聲音與情緒的連結', '培養超人氣的三大表達句型：讚美、應答、換句話說', '提升人際互動好感度三部曲：電話溝通、網上溝通、碰面溝通', '說話不是一門藝術，而是可以學習的技術！用心練表達，讓世界聽見你的聲音！']
['Beforeyoustart:tokeepinmind', 'Structure1-styleofapersonalstatement', 'Structure2-Keyhol

In [6]:
def get_course_chapters(x):
    if x["course_id"] in course_chapters.keys():
        return str(course_chapters[x["course_id"]])
    else:
        return ""

course_df['chapters'] = course_df.apply(get_course_chapters, axis=1)


In [10]:
def clean_data(x):
    return x.replace(" ", "")

filledna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'groups', 'sub_groups', 'topics', 'will_learn', 'recommended_background', 'target_group', 'required_tools', 'chapters']
filledna=filledna[features]
for feature in features:
    filledna[feature] = filledna[feature].apply(clean_data)
    
filledna.head(5)

,course_name,teacher_intro,groups,sub_groups,topics,will_learn,recommended_background,target_group,required_tools,chapters
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,只要你有一顆愛化妝、想變漂亮的心皆可以參加。\n\n⚠️雖然課程當中會帶到相關彩妝技巧，不過...,熱愛彩妝的人,所需工具為：視課程實際會用到的彩妝用品,
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,知道如何使用Illustrator的基本工具列,每一位興趣的人都能學得來，非常容易的小技巧,"AdobeIllustrator（必備）,camera","['拉曲線','分割工具','油漆桶上色工具','幾何圖形分割X色塊組合']"
2,數位拼貼的手感,自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、插畫製作等，偶爾...,"藝術,設計","平面設計,電腦繪圖",photoshop,藉由這堂課，可以學習到如何利用身邊的物件自製素材，了解去背、Photoshop的基本操作，並...,,繪圖軟體初學者；喜歡隨性創作；常天馬行空；喜歡剪剪貼貼或收集東西，卻不知該如何收藏。,軟體：AdobePhotoshop、AdobeIllustrator；\n設備：電腦、相機、...,"['作品範例','素材來源與製作','去背方式','畫面配置-背景與主題','畫面配置-細節..."
3,Line的貼圖自己動手做！,我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物設計、AppUI設計，201...,"藝術,設計","平面設計,應用設計,電腦繪圖","LINE貼圖設計,photoshop",如果上完課之後好好練習(不懂的地方隨時提問)，就可以畫出你在一般市面上看的到的line貼圖(...,只要手可以拿得起鉛筆和握滑鼠就可以,喜歡畫畫，但是沒有學過電腦繪圖的人、想學基本的電腦繪圖但不知如何入手的人，從10歲-100歲...,Photoshop(可至官網下載試用版),"['課外延伸閱讀(必看)','Photoshop基本功能說明-1','Photoshop基本..."
4,為申請學校或工作寫好英文自傳,在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研細胞奈米科學與皮膚再生醫學的過...,"職場技能,語言","求職,英文","商用英文,履歷撰寫","Apersonalstatementthatyoucanbeproudof,andarewa...",HowtouseGoogleDoc.BasicEnglishlisteningcompreh...,Peopleapplyingforcollege/post-graduatestudieso...,GoogleDocwillbeusedtosubmit/shareyourassignmen...,"['Beforeyoustart:tokeepinmind','Structure1-sty..."


In [32]:
import jieba.analyse
def create_soup(x):
    #course_name = str(jieba.analyse.extract_tags(x['course_name'], 10))
    # teacher_intro = str(jieba.analyse.extract_tags(x['teacher_intro'], 10))
    # will_learn = str(jieba.analyse.extract_tags(x['will_learn'], 10))
    # recommended_background = str(jieba.analyse.extract_tags(x['recommended_background'], 10))
    #target_group = str(jieba.analyse.extract_tags(x['target_group'], 10))
    # required_tools = str(jieba.analyse.extract_tags(x['required_tools'], 10))
    chapters = str(jieba.analyse.extract_tags(x['chapters'], 20))
    course_name = x['course_name']
    teacher_intro = x['teacher_intro']
    will_learn = x['will_learn']
    recommended_background = x['recommended_background']
    target_group = x['target_group']
    required_tools = x['required_tools']
    #chapters = x['chapters']
    text = course_name + ' ' + teacher_intro + ' ' + x['groups'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + will_learn + ' ' + recommended_background + ' ' + target_group  + ' ' + chapters
    return text

filledna['soup'] = filledna.apply(create_soup, axis=1)
filledna['soup'].head(5)

0    少女人妻華麗變身：七大妝容七彩的夢幻樂園 在美妝KOL圈裡屬個人風格強烈的Alice，在清新...
1    幾何圖形分割X色塊組合 從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現...
2    數位拼貼的手感 自由工作者，致力於品牌視覺規劃，包含識別系統設計、平面設計、包裝設計、網頁、...
3    Line的貼圖自己動手做！ 我是Danny，現是一名自由接案的插畫師，主要是做插畫設計、人物...
4    為申請學校或工作寫好英文自傳 在北美長大，回到南台灣尋根後，現落腳於南加州的庄腳嬰仔。在鑽研...
Name: soup, dtype: object

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import datetime
from scipy.sparse import csr_matrix

In [33]:
tfidf = TfidfVectorizer(stop_words= None)
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(filledna['soup'])

#print(tfidf.vocabulary_)
tfidf_matrix_nd = tfidf_matrix.toarray()
#np.save('item_embeddings_.npy', tfidf_matrix_nd)

In [34]:
print(len(tfidf.get_feature_names_out()))
#tfidf.vocabulary_ 

29808


{'少女人妻華麗變身': 12324,
 '七大妝容七彩的夢幻樂園': 2701,
 '在美妝kol圈裡屬個人風格強烈的alice': 9555,
 '在清新與叛逆風格間遊刃有餘': 9525,
 '其幽默的美妝影片手法引起許多粉絲的共鳴': 6406,
 '因此廣受廠商與粉絲喜愛': 9158,
 '生活品味': 21227,
 '更多生活品味': 18000,
 '護膚保養與化妝': 26135,
 '不再害怕各種顏色的彩妝': 2910,
 '可以更隨心搭配各種繽紛的顏色': 8120,
 '只要你有一顆愛化妝': 7871,
 '想變漂亮的心皆可以參加': 15020,
 '雖然課程當中會帶到相關彩妝技巧': 28884,
 '不過內容偏向切入': 3241,
 '色系使用': 24563,
 '因此會建議對彩妝有基礎認識的同學較好上手': 9167,
 '再請同學參考': 6542,
 '熱愛彩妝的人': 20698,
 '幾何圖形分割x色塊組合': 13129,
 '從學生時代開始摸索photoshop等軟體': 13691,
 '自以為有些天賦但後來發現其實沒有': 24227,
 '出社會後從事美術編輯一類的工作': 6612,
 '因而接觸了印刷方面還有版面設計的知識': 9232,
 '工作了一段時間後發現自己對於設計即將失去熱誠': 12496,
 '分析的結果': 6696,
 '原因可能是太頻繁的接觸單一領域': 7610,
 '所以開始隨意的畫畫': 15921,
 '隨意的學習軟體技巧': 28828,
 '試著從中找回熱誠': 25629,
 '後來發現這樣的方式是有用的': 13538,
 '無論何事': 20531,
 '熱誠總是最重要的': 20723,
 '在設計的路上': 9583,
 '大家都是過客': 10093,
 '需要面對的': 29171,
 '需要克服的': 29094,
 '永遠是自己': 19763,
 'www': 2358,
 'behance': 542,
 'net': 1608,
 'jafyxxx': 1278,
 '藝術': 24886,
 '設計': 25496,
 '平面設計': 13096,
 '繪畫與插畫': 23484,
 'illustrator': 1216,
 '以拉': 48

In [38]:
from scipy.sparse import csr_matrix
item_embeddings_FM = np.load('lightFM_embeding/item_embeddings_50.npy')
item_embeddings_FM = item_embeddings_FM[:728]
print(tfidf_matrix.shape)
print(item_embeddings_FM.shape)

(728, 29808)
(728, 50)


In [44]:
concatenate_item_embeddings_nd = np.concatenate([tfidf_matrix_nd, item_embeddings_FM/50], axis = 1)
print(concatenate_item_embeddings_nd.shape)
concatenate_item_embeddings = csr_matrix(concatenate_item_embeddings_nd)
print(concatenate_item_embeddings[0])

(728, 29858)
  (0, 2701)	0.22599171793277803
  (0, 2910)	0.22599171793277803
  (0, 3241)	0.22599171793277803
  (0, 6406)	0.22599171793277803
  (0, 6542)	0.22599171793277803
  (0, 7871)	0.22599171793277803
  (0, 8120)	0.22599171793277803
  (0, 9158)	0.22599171793277803
  (0, 9167)	0.22599171793277803
  (0, 9525)	0.22599171793277803
  (0, 9555)	0.22599171793277803
  (0, 12324)	0.22599171793277803
  (0, 15020)	0.22599171793277803
  (0, 18000)	0.2228963272815302
  (0, 20698)	0.22599171793277803
  (0, 21227)	0.09078919564634716
  (0, 24563)	0.22599171793277803
  (0, 26135)	0.3534380342449886
  (0, 28884)	0.22599171793277803
  (0, 29808)	0.003565486054867506
  (0, 29809)	0.003164491383358836
  (0, 29810)	0.0030265511013567448
  (0, 29811)	-0.003175850259140134
  (0, 29812)	-0.0018263979582116008
  (0, 29813)	-0.003372777486220002
  :	:
  (0, 29833)	-0.0032861558720469475
  (0, 29834)	0.003350535873323679
  (0, 29835)	-0.003298740368336439
  (0, 29836)	-0.0032145935110747814
  (0, 29837)	0.00

In [45]:

# Compute the cosine similarity matrix
print(concatenate_item_embeddings_nd.shape)
cosine_sim2 = linear_kernel(concatenate_item_embeddings, concatenate_item_embeddings)
#Output the shape of tfidf_matrix
cosine_sim2.shape


# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(filledna['soup'])

# cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

(728, 29858)


(728, 728)

In [41]:
import pickle
tfidf_save = {"TfidfVectorizer": tfidf, "Tfidf_matrix": tfidf_matrix, "lightFM_embeding_matrix": item_embeddings_FM, "concatenate_matrix": concatenate_item_embeddings}
with open('content_embeding/all_2.pickle', 'wb') as f:
    pickle.dump(tfidf_save, f)

# 根據課程相似度 Predict Subgroups

In [63]:
subgroups_df = pd.read_csv('./data/subgroups.csv')
subgroup_num = len(subgroups_df)

subgroups2idx = {}
for (id, name) in zip(subgroups_df["subgroup_id"], subgroups_df["subgroup_name"]):
    subgroups2idx.update({name: id})
idx2subgroups = {v : k for k, v in subgroups2idx.items()}

course2subgroups = {}
for (course_id, sub_groups) in zip(course_df["course_id"], course_df["sub_groups"]):
    if pd.isnull(sub_groups):
        course2subgroups.update({course_id: [0]})
    else:
        course2subgroups.update({course_id: [ subgroups2idx[sub_group] for sub_group in sub_groups.split(',')]})

course2subgroups

{'61888e868f154b000781b45a': [1, 2],
 '54d5a117065a7e0e00725ac0': [3, 4],
 '54d5d9952246e60a009ec571': [3, 5],
 '54d7148a2246e60a009ec588': [3, 6, 5],
 '5513e92b38239d10005778e1': [7, 8],
 '55307d94d530a90a00a3a896': [9],
 '551a6be023774e0a001eb20c': [4],
 '551171a938239d1000577864': [10, 11],
 '54f1268f4ec3c809002e4a29': [12],
 '55599687dfe21b0a00e776d9': [13],
 '559e49185850311000fca504': [14, 1, 15],
 '55a4d6397b4d99100011957d': [13],
 '55ae208a7b4d9910001198f2': [1, 16],
 '55b0ba667b4d991000119a92': [15],
 '557ad62bd736230f00adb2ab': [17],
 '556ad58777a8710900bedffe': [18],
 '55ae36c87b4d99100011990f': [15],
 '55ae34247b4d99100011990d': [15],
 '55ae66017b4d991000119959': [15],
 '5591501e6dec460f00111314': [4],
 '55c636d27b4d99100011a4aa': [19],
 '5593f992cfe8320b00ccd4c4': [6, 20, 4],
 '559a217dcfe8320b00ccd75b': [21],
 '55e3dccbfa223d100058f3cb': [4],
 '5606262ac61d930a00455fad': [4],
 '55e82d8efa223d100058f6f4': [3, 5],
 '55e66f33fa223d100058f5f7': [1],
 '55ea1ea8fa223d100058f7df

In [84]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 10):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_subgroups = []
    # Get the scores of the top most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_subgroups) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                course_id = course_df['course_id'][sim_scores[i][0]]
                for subgroup in course2subgroups[course_id]:
                    if subgroup not in recommend_subgroups:
                        recommend_subgroups.append(subgroup)

    return recommend_subgroups

test = ['5513e92b38239d10005778e1']
print(course_df['course_name'].iloc[[course2id_mapping[course] for course in test]].tolist())  
test_predict = get_recommendations_new(test, cosine_sim2, 20)
[ idx2subgroups[subgroup_id] for subgroup_id in test_predict ]

['為申請學校或工作寫好英文自傳']


['職場溝通',
 '英文',
 '求職',
 '個人品牌經營',
 '更多職場技能',
 '網站架設',
 '更多生活品味',
 '親子教育',
 '平面設計',
 '電腦繪圖',
 '日文',
 '心靈成長與教育',
 '更多語言',
 '歐洲語言',
 '創業',
 '數位行銷',
 '社群行銷',
 '角色設計',
 '資料彙整',
 '程式入門',
 '程式語言',
 '網頁前端']

In [71]:
seen_predict_df = pd.read_csv('./data/test_seen_group.csv')
predict_user = seen_predict_df["user_id"].to_list()
print(len(predict_user))


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

7205
59737


# Predict and Save

In [85]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("subgroup_predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "subgroup"])
    for user_id in predict_users:
        recommend_subgroups_str = [ str(x) for x in get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 50)]
        recommend = " ".join(recommend_subgroups_str)
        writer.writerow([user_id, recommend])

# Map50

In [102]:
val_seen_group_df["subgroup"].isna().sum()

33

In [99]:
train_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    train_user_haved_purchased_course[seen_user_id] = train_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list

val_answer_subgroups = {}
val_seen_group_df_filledna = val_seen_group_df.fillna("")
for seen_user_id, subgroup_ids in zip(val_seen_group_df_filledna["user_id"], val_seen_group_df_filledna["subgroup"]):
    if len(subgroup_ids) > 0:
        course_id_list = [ int(x) for x in subgroup_ids.split(' ')]
        val_answer_subgroups[seen_user_id] = val_answer_subgroups.setdefault(seen_user_id, []) + course_id_list
val_answer_subgroups

{'56dae2b74e3ef90900b7bd0e': [37, 71],
 '60e66f29be3e3b0006c4db75': [1, 14, 15],
 '5c919efb728ddf00208b9b2b': [1, 3, 5, 25, 29, 50, 51, 52, 66, 67, 72, 73],
 '5ac115507997a2001e7c3617': [33, 34, 38, 39],
 '5f53b84440c5be3bb873a9d3': [30],
 '5be70b19870dcd00200c5b43': [1, 12, 29, 50, 51, 61, 63, 64],
 '5c8092eea52fc8002453f113': [1,
  6,
  7,
  29,
  33,
  34,
  36,
  38,
  39,
  50,
  51,
  54,
  61,
  66,
  72],
 '5c6f5e6dab456a001f6ec98b': [32, 47, 48, 52],
 '60d1adf755f61e06bf743e94': [69],
 '5c2e4d016bbdc8002101d3d1': [59, 71],
 '5cc96d6d4557c300204723ff': [5],
 '578da08ad0c3030700416fbf': [69],
 '5f65e2524e1c4647b0e57336': [3, 5, 21, 36, 49],
 '5c637ad06ddef70020c7c728': [58],
 '5f18f3323ad0ad42a8eb3ac2': [1,
  3,
  4,
  5,
  7,
  28,
  29,
  33,
  36,
  37,
  40,
  50,
  51,
  59,
  60,
  61,
  64,
  66,
  69,
  72,
  75,
  76,
  82,
  83,
  84,
  85],
 '5aa3f67a5d786f001e632547': [3, 5, 6, 51, 54, 67],
 '5e84fab7a56f8200253ab2ca': [3, 5, 6, 12, 51, 54, 63, 64, 67],
 '5ef5bbe6303

In [100]:
import ml_metrics

answers = []
predictions = []
map50s = []
for user_id in val_seen_group_df["user_id"]:
    prediction = get_recommendations_new(train_user_haved_purchased_course[user_id], cosine_sim2, top = 10)

    prediction_idxs = prediction
    if user_id in val_answer_subgroups.keys():
        answer_idxs = val_answer_subgroups[user_id]
    else:
        answer_idxs = []

    print(user_id)
    print([course2subgroups[x] for x in train_user_haved_purchased_course[user_id]])
    print(prediction) 
    print(answer_idxs)
    
    print("")
    predictions.append(prediction_idxs)
    answers.append(answer_idxs)
    map50s.append(ml_metrics.mapk(predicted= [prediction_idxs], actual= [answer_idxs], k = 50))
    
print(np.mean(map50s))
#answers.append(answers)        

map50 = ml_metrics.mapk(predicted= predictions, actual= answers, k = 50)
map50

56dae2b74e3ef90900b7bd0e
[[71, 59], [83, 37], [7], [8], [51]]
[50, 61, 29, 66, 1, 51, 7, 72, 36, 8]
[37, 71]

60e66f29be3e3b0006c4db75
[[1, 16]]
[1, 16, 8, 25, 6, 2, 59, 60, 46, 32, 33]
[1, 14, 15]

5c919efb728ddf00208b9b2b
[[40], [40], [71, 59], [57], [59], [8], [38, 51, 54, 34, 39, 67], [32, 33, 58], [59, 8], [59], [50, 61, 29, 66, 1, 51, 7, 72, 36], [52, 57], [40], [8], [8], [52], [57], [51, 54], [38, 34, 39], [51], [3, 5], [59], [33, 57]]
[40, 29, 1, 57, 3, 5, 33, 81, 67, 59, 8]
[1, 3, 5, 25, 29, 50, 51, 52, 66, 67, 72, 73]

5ac115507997a2001e7c3617
[[7], [33, 34], [23, 3, 6]]
[40, 50, 51, 5, 32, 33, 34, 6, 25, 7]
[33, 34, 38, 39]

5f53b84440c5be3bb873a9d3
[[30]]
[30, 10, 11, 21, 40, 84, 50, 53, 51, 72]
[30]

5be70b19870dcd00200c5b43
[[63, 75, 33, 76, 64], [75, 33, 76, 64], [77, 78], [75, 76], [71, 59], [33, 34]]
[69, 70, 32, 75, 76, 81, 64, 35, 57, 33, 34]
[1, 12, 29, 50, 51, 61, 63, 64]

5c8092eea52fc8002453f113
[[71, 59], [78, 59], [71, 59], [59], [59], [19], [51], [59], [59]]
[

KeyError: '5ee6472de2246096f7d4e1a6'

In [ ]:
No//
無
0.06292329189817554
+ tool, chapter
0.06052279089383824
+ tool, 
0.06050695911579399
+ chapter
0.061850422031892574

jieba analyse all
0.05520367812254581
jieba chapter
0.064379711888704
jieba course_name, target, chapter
0.06359486355498518
50//
25
0.06292390839334439
10:
0.06567479525066472
0:
0.039646865330987505
5:
0.058353096023023646
8:
0.06538277231089117
9:
0.06582550600807187
15:
0.0638514124150609

10:
0.06682009120032709

11 jieba course_name, target, chapter:
0.06747995491470742
10 jieba course_name, target, chapter:
0.06766655840317758
25 jieba course_name, target, chapter:
0.06400045804229197
15 jieba course_name, target, chapter:
0.06519274413773107

In [ ]:
100//
9:
0.06548068054070211
5:
0.058060431797675165
15:
0.0642605326969306
8:
0.06423974137947344
11:
0.06564594757330318
13:
0.06483003396812996
12:
0.0651953754497303